In [ ]:
# Load environment variables from parent directory and set up auto-reload
import os

from dotenv import load_dotenv

load_dotenv(os.path.join("..", ".env"), override=True)

%load_ext autoreload
%autoreload 2

#TODO(Geoff): Here we can import all components and create the full agent w/ research tool.

In [ ]:
from typing import Literal

from langchain_core.tools import tool
from tavily import TavilyClient


# Create the Tavily search tool
@tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search using Tavily."""
    tavily_client = TavilyClient()  # Studio handles API key automatically
    search_results = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_results

#TODO(Geoff): We can show Studio here.